# 🚀 EchoNote Dataset Generator v4

**Diversity-Focused Architecture**

Key improvements:
- **Scenario seeds** - Unique random elements for each generation
- **NO examples in prompt** - Prevents copying
- **Diverse openings** - 20+ different meeting start patterns
- **Meeting dynamics** - Varying energy, conflict, formality
- **Speaker personas** - Different communication styles
- **Chain-of-Thought** - Model plans before generating

In [ ]:
# Install dependencies
!pip install -q vllm>=0.6.0 jedi>=0.16
!pip install -q spacy>=3.7.0 textblob>=0.18.0
!pip install -q jsonschema>=4.21.0 tqdm>=4.66.0
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json
import re
import random
import time
import hashlib
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, field
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm
import spacy
from textblob import TextBlob
from jsonschema import validate, ValidationError

print("Imports complete.")

Imports complete.


In [ ]:
from vllm import LLM, SamplingParams

print("Loading Qwen2.5-7B-Instruct...")

llm = LLM(
    model="Qwen/Qwen2.5-7B-Instruct",
    trust_remote_code=True,
    max_model_len=8192,
    gpu_memory_utilization=0.95,
    dtype="half"
)

print("✅ Model loaded!")

Loading Qwen2.5-7B-Instruct...
INFO 12-22 22:31:31 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'half', 'max_model_len': 8192, 'gpu_memory_utilization': 0.95, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-7B-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

INFO 12-22 22:31:50 [model.py:514] Resolved architecture: Qwen2ForCausalLM
WARNING 12-22 22:31:50 [model.py:2005] Casting torch.bfloat16 to torch.float16.
INFO 12-22 22:31:50 [model.py:1661] Using max model len 8192
INFO 12-22 22:31:50 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

WARNING 12-22 22:31:53 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-22 22:34:01 [llm.py:360] Supported tasks: ['generate']
✅ Model loaded!


In [ ]:
nlp = spacy.load("en_core_web_lg")
print("NLP loaded.")

NLP loaded.


## Configuration

In [ ]:
@dataclass
class Config:
    total_samples: int = 500
    batch_size: int = 16

    transcript_max_tokens: int = 2200
    output_max_tokens: int = 1000

    min_transcript_words: int = 700
    max_transcript_words: int = 2200

    min_summary_words: int = 60
    max_summary_words: int = 100

    sentiment_distribution: Dict[str, float] = field(default_factory=lambda: {
        "positive": 0.60, "neutral": 0.25, "negative": 0.15
    })

    meeting_types: List[str] = field(default_factory=lambda: [
        "quarterly_review", "performance_review", "sales_call", "project_kickoff",
        "board_meeting", "team_standup", "strategy_session", "product_planning",
        "investor_update", "client_meeting", "one_on_one", "retrospective",
        "budget_review", "hiring_discussion", "incident_postmortem", "training_session"
    ])

config = Config()
print(f"Config: {config.total_samples} samples, {config.min_transcript_words}-{config.max_transcript_words} words")

Config: 500 samples, 700-2200 words


In [ ]:
OUTPUT_SCHEMA = {
    "type": "object",
    "required": ["executiveSummary", "keyDecisions", "actionItems", "nextSteps", "keyTopics", "sentiment"],
    "properties": {
        "executiveSummary": {"type": "string", "minLength": 150},
        "keyDecisions": {"type": "array", "items": {"type": "string"}, "minItems": 2, "maxItems": 6},
        "actionItems": {
            "type": "array",
            "items": {
                "type": "object",
                "required": ["task", "assignee", "deadline", "priority"],
                "properties": {
                    "task": {"type": "string", "minLength": 10},
                    "assignee": {"type": "string"},
                    "deadline": {"type": "string"},
                    "priority": {"type": "string", "enum": ["high", "medium", "low"]}
                }
            },
            "minItems": 2,
            "maxItems": 6
        },
        "nextSteps": {"type": "array", "items": {"type": "string"}, "minItems": 2, "maxItems": 5},
        "keyTopics": {"type": "array", "items": {"type": "string"}, "minItems": 3, "maxItems": 7},
        "sentiment": {"type": "string", "enum": ["positive", "neutral", "negative"]}
    }
}

## Diversity Seeds - The Key to Unique Transcripts

In [ ]:
# OPENING STYLES - How meetings start (25+ variations)
OPENING_STYLES = [
    "jumping straight into the agenda",
    "with casual small talk about the weekend before getting to business",
    "by addressing an urgent issue that came up this morning",
    "with a quick recap of last meeting's action items",
    "by welcoming a new team member who just joined",
    "with technical difficulties and people joining late",
    "in the middle of an ongoing heated discussion",
    "with someone sharing exciting news before the formal agenda",
    "by reviewing the meeting objectives",
    "with the leader apologizing for running behind schedule",
    "by celebrating a recent win",
    "with concerns about a competitor's announcement",
    "by addressing team morale issues",
    "with an unexpected guest joining the call",
    "by going around the room for quick status updates",
    "with someone presenting their screen",
    "by reading out a customer complaint",
    "with a moment of silence for a difficult topic",
    "by brainstorming ideas on a whiteboard",
    "with a demo of the latest product build",
    "by reviewing metrics on a dashboard",
    "with jokes and laughter before settling down",
    "by passing around a document everyone needs to review",
    "with someone dialing in from an unusual location",
    "by addressing rumors that have been circulating"
]

# MEETING DYNAMICS - Energy and conflict levels
MEETING_DYNAMICS = [
    {"energy": "high", "conflict": "none", "desc": "enthusiastic and aligned, everyone agrees"},
    {"energy": "high", "conflict": "productive", "desc": "energetic debate with constructive disagreements"},
    {"energy": "high", "conflict": "tense", "desc": "heated arguments and frustration"},
    {"energy": "low", "conflict": "none", "desc": "calm and routine, business as usual"},
    {"energy": "low", "conflict": "underlying", "desc": "subdued with unspoken tensions"},
    {"energy": "mixed", "conflict": "none", "desc": "some people engaged, others distracted"},
    {"energy": "mixed", "conflict": "resolved", "desc": "started tense but reached resolution"},
    {"energy": "building", "conflict": "none", "desc": "starts slow but builds excitement"},
    {"energy": "declining", "conflict": "fatigue", "desc": "running long, people getting tired"}
]

# SPEAKER PERSONAS - Communication styles
SPEAKER_PERSONAS = [
    {"style": "data-driven", "speech": "precise numbers, statistics, analytical language"},
    {"style": "visionary", "speech": "big picture thinking, inspirational, future-focused"},
    {"style": "pragmatic", "speech": "practical, action-oriented, cuts to the chase"},
    {"style": "diplomatic", "speech": "careful wording, considers all perspectives, mediates"},
    {"style": "technical", "speech": "detailed explanations, jargon, deep dives"},
    {"style": "casual", "speech": "informal, uses humor, relatable analogies"},
    {"style": "formal", "speech": "professional, structured, by-the-book"},
    {"style": "skeptical", "speech": "questions assumptions, plays devil's advocate"},
    {"style": "enthusiastic", "speech": "excited, supportive, cheerleader energy"},
    {"style": "concise", "speech": "brief, to the point, dislikes rambling"}
]

# SPECIFIC SITUATIONS - Unique scenarios
SITUATIONS = [
    "discussing whether to pivot the product strategy",
    "reviewing why a major deal fell through",
    "planning the response to a PR crisis",
    "debating which features to cut before launch",
    "celebrating hitting a major milestone",
    "addressing a team member's underperformance",
    "deciding on budget allocation for next quarter",
    "reviewing customer churn analysis",
    "planning a product launch timeline",
    "discussing acquisition rumors",
    "reviewing security audit findings",
    "planning headcount for next year",
    "analyzing competitor's new feature",
    "discussing remote work policy changes",
    "reviewing failed A/B test results",
    "planning company offsite",
    "discussing technical debt priorities",
    "reviewing customer NPS feedback",
    "planning integration with a partner",
    "discussing data privacy compliance",
    "reviewing sprint velocity trends",
    "planning marketing campaign",
    "discussing vendor contract renewal",
    "reviewing support ticket backlog",
    "planning beta program expansion"
]

# INTERRUPTION PATTERNS - Natural conversation flow
INTERRUPTION_PATTERNS = [
    "someone's phone rings and they apologize",
    "connection issues cause someone to drop and rejoin",
    "side conversation happens that others can hear",
    "someone gets an urgent Slack message",
    "background noise from someone's location",
    "someone needs to leave early and hands off",
    "a quick tangent about an unrelated topic",
    "someone asks to repeat what was just said",
    "crosstalk where two people speak at once",
    "awkward silence that someone fills"
]

print(f"Diversity seeds loaded:")
print(f"  Opening styles: {len(OPENING_STYLES)}")
print(f"  Meeting dynamics: {len(MEETING_DYNAMICS)}")
print(f"  Speaker personas: {len(SPEAKER_PERSONAS)}")
print(f"  Situations: {len(SITUATIONS)}")
print(f"  Interruptions: {len(INTERRUPTION_PATTERNS)}")

Diversity seeds loaded:
  Opening styles: 25
  Meeting dynamics: 9
  Speaker personas: 10
  Situations: 25
  Interruptions: 10


In [ ]:
# COMPANIES with industry context
COMPANIES = [
    ("TechVentures", "B2B SaaS"), ("CloudNine Solutions", "cloud infrastructure"),
    ("DataPrime Analytics", "data analytics"), ("NexGen Systems", "enterprise software"),
    ("FinEdge Capital", "fintech"), ("PayStream", "payments"),
    ("MedTech Innovations", "healthcare tech"), ("BioGenesis Labs", "biotech"),
    ("ShopifyPlus Partners", "e-commerce"), ("RetailEdge", "retail tech"),
    ("Jetpack Labs", "consumer app startup"), ("Moonshot Inc", "deep tech startup"),
    ("Accel Consulting", "management consulting"), ("Insight Partners", "advisory"),
    ("AdVantage Media", "digital advertising"), ("BrandBoost", "marketing tech"),
    ("Precision Manufacturing", "industrial IoT"), ("AutomaTech Systems", "robotics"),
    ("EduLearn Platform", "edtech"), ("SkillUp Academy", "corporate training"),
    ("GreenEnergy Solutions", "cleantech"), ("SolarFlow", "renewable energy"),
    ("SecureNet", "cybersecurity"), ("DataGuard", "privacy compliance"),
    ("TravelWise", "travel tech"), ("BookingEngine", "hospitality"),
    ("FoodTech Labs", "food delivery"), ("QuickBite", "restaurant tech"),
    ("PropTech Solutions", "real estate tech"), ("HomeBase", "property management")
]

FIRST_NAMES = [
    "James", "Sarah", "Michael", "Emily", "David", "Jennifer", "Robert", "Lisa",
    "William", "Amanda", "Chris", "Jessica", "Daniel", "Ashley", "Matthew", "Rachel",
    "Wei", "Ming", "Priya", "Raj", "Carlos", "Maria", "Ahmed", "Fatima",
    "Andrei", "Natasha", "Pierre", "Claire", "Marcus", "Jasmine", "Kenji", "Yuki",
    "Emma", "Olivia", "Liam", "Noah", "Sophia", "Aiden", "Isabella", "Lucas",
    "Sanjay", "Meera", "Diego", "Sofia", "Omar", "Layla", "Ivan", "Olga"
]

LAST_NAMES = [
    "Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Miller", "Davis",
    "Chen", "Wang", "Li", "Zhang", "Patel", "Shah", "Kumar", "Singh",
    "Kim", "Park", "Tanaka", "Yamamoto", "Mueller", "Schmidt", "Dubois", "Moreau",
    "O'Brien", "Murphy", "Kelly", "Sullivan", "Rodriguez", "Martinez", "Lopez", "Gonzalez",
    "Thompson", "White", "Harris", "Clark", "Lewis", "Walker", "Hall", "Young"
]

ROLES = [
    "CEO", "CTO", "CFO", "COO", "VP of Engineering", "VP of Sales", "VP of Product",
    "Director of Engineering", "Director of Product", "Director of Marketing",
    "Engineering Manager", "Product Manager", "Project Manager", "Program Manager",
    "Tech Lead", "Senior Engineer", "Staff Engineer", "Principal Engineer",
    "Data Scientist", "UX Designer", "Account Executive", "Customer Success Manager",
    "Solutions Architect", "DevOps Lead", "QA Manager", "Scrum Master"
]

DEADLINES = [
    "end of day", "by tomorrow", "by Wednesday", "by Friday", "end of week",
    "next Monday", "early next week", "within a week", "within two weeks",
    "by the 15th", "end of month", "by end of sprint", "next sprint",
    "before the launch", "before the board meeting", "by end of quarter"
]

def random_name():
    return f"{random.choice(FIRST_NAMES)} {random.choice(LAST_NAMES)}"

def random_participants(n=3):
    participants = []
    used_names = set()
    personas = random.sample(SPEAKER_PERSONAS, min(n, len(SPEAKER_PERSONAS)))

    for i in range(n):
        name = random_name()
        while name in used_names:
            name = random_name()
        used_names.add(name)
        participants.append({
            "name": name,
            "role": random.choice(ROLES),
            "persona": personas[i] if i < len(personas) else random.choice(SPEAKER_PERSONAS)
        })
    return participants

print(f"Data pools: {len(COMPANIES)} companies, {len(FIRST_NAMES)*len(LAST_NAMES)} names")

Data pools: 30 companies, 1920 names


## Scenario Seed Generator

In [ ]:
def generate_scenario_seed(meeting_type: str, sentiment: str) -> Dict:
    """Generate a unique scenario seed with random elements for diversity"""

    company, industry = random.choice(COMPANIES)
    participants = random_participants(random.randint(2, 4))

    # Random metrics with specific values
    metrics = {}
    metric_types = random.sample([
        ("revenue", f"${random.randint(1, 50)}M"),
        ("MRR", f"${random.randint(100, 900)}K"),
        ("ARR", f"${random.randint(1, 20)}M"),
        ("growth rate", f"{random.randint(5, 150)}%"),
        ("churn rate", f"{random.randint(1, 15)}%"),
        ("NPS score", f"{random.randint(20, 80)}"),
        ("conversion rate", f"{random.randint(2, 25)}%"),
        ("DAU", f"{random.randint(10, 500)}K"),
        ("customer count", f"{random.randint(50, 5000)}"),
        ("deal pipeline", f"${random.randint(1, 10)}M"),
        ("burn rate", f"${random.randint(100, 800)}K/month"),
        ("runway", f"{random.randint(6, 24)} months"),
        ("headcount", f"{random.randint(20, 500)} people")
    ], k=random.randint(2, 4))
    for k, v in metric_types:
        metrics[k] = v

    # Sentiment-appropriate outcomes
    if sentiment == "positive":
        outcome = random.choice([
            "exceeded quarterly targets by 20%",
            "closed a major enterprise deal",
            "successful product launch with great reception",
            "team hit all sprint goals ahead of schedule",
            "received Series B funding commitment",
            "customer satisfaction scores at all-time high",
            "competitor's failure created market opportunity"
        ])
    elif sentiment == "negative":
        outcome = random.choice([
            "missed revenue target by 30%",
            "lost key customer to competitor",
            "product launch delayed by 2 months",
            "two senior engineers resigned",
            "security incident affecting customer data",
            "failed to close expected funding round",
            "major bug discovered in production"
        ])
    else:
        outcome = random.choice([
            "met expectations with room for improvement",
            "mixed results across different metrics",
            "steady progress on roadmap items",
            "some wins and some setbacks balanced out",
            "on track but facing some headwinds",
            "standard quarterly performance"
        ])

    # Create unique seed
    seed = {
        "company": company,
        "industry": industry,
        "participants": participants,
        "meeting_type": meeting_type,
        "sentiment": sentiment,
        "metrics": metrics,
        "outcome": outcome,
        "opening_style": random.choice(OPENING_STYLES),
        "dynamics": random.choice(MEETING_DYNAMICS),
        "situation": random.choice(SITUATIONS),
        "interruption": random.choice(INTERRUPTION_PATTERNS) if random.random() > 0.5 else None,
        "unique_id": hashlib.md5(f"{time.time()}{random.random()}".encode()).hexdigest()[:8]
    }

    return seed

# Test
test_seed = generate_scenario_seed("quarterly_review", "positive")
print("Sample scenario seed:")
print(json.dumps({k: v for k, v in test_seed.items() if k != 'participants'}, indent=2))

Sample scenario seed:
{
  "company": "Precision Manufacturing",
  "industry": "industrial IoT",
  "meeting_type": "quarterly_review",
  "sentiment": "positive",
  "metrics": {
    "burn rate": "$523K/month",
    "runway": "21 months",
    "ARR": "$6M",
    "headcount": "46 people"
  },
  "outcome": "successful product launch with great reception",
  "opening_style": "by welcoming a new team member who just joined",
  "dynamics": {
    "energy": "declining",
    "conflict": "fatigue",
    "desc": "running long, people getting tired"
  },
  "situation": "addressing a team member's underperformance",
  "interruption": null,
  "unique_id": "d745d6a0"
}


## Transcript Prompt - NO EXAMPLES

In [ ]:
def create_transcript_prompt(seed: Dict) -> str:
    """Create prompt WITHOUT examples - uses scenario seed for diversity"""

    names = [p['name'] for p in seed['participants']]
    personas = [f"{p['name']} ({p['role']}, {p['persona']['style']} communicator)" for p in seed['participants']]
    metrics_str = ", ".join([f"{k}: {v}" for k, v in seed['metrics'].items()])

    # Build persona descriptions
    persona_desc = "\n".join([f"- {p['name']}: {p['persona']['speech']}" for p in seed['participants']])

    prompt = f"""Generate a raw Whisper ASR transcription of a {seed['meeting_type'].replace('_', ' ')} meeting.

SCENARIO SEED #{seed['unique_id']}:
Company: {seed['company']} ({seed['industry']})
Participants: {', '.join(names)}
Key metrics: {metrics_str}
Main topic: {seed['situation']}
Outcome: {seed['outcome']}
Meeting energy: {seed['dynamics']['desc']}

SPEAKER COMMUNICATION STYLES:
{persona_desc}

MEETING FLOW:
1. Opens {seed['opening_style']}
2. Discussion of {seed['situation']} with specific metrics
3. {f"Includes: {seed['interruption']}" if seed['interruption'] else "Flows naturally without major interruptions"}
4. Ends with action items assigned to specific people

ASR TRANSCRIPTION RULES:
- Output ONLY the transcribed speech as continuous paragraphs
- NO speaker labels (no "Name:" or "Speaker 1:")
- Names appear naturally IN speech: "thanks {names[0]}", "{names[-1]} what about"
- Include natural speech: pauses, self-corrections, thinking aloud
- Each speaker's style should reflect their persona above
- Reference the SPECIFIC metrics and numbers provided

LENGTH: Write 1000-1400 words of continuous speech.

BEGIN TRANSCRIPTION:"""

    return prompt

print("Transcript prompt generator ready.")

Transcript prompt generator ready.


In [ ]:
def create_output_prompt(transcript: str, participants: List[Dict], target_sentiment: str) -> str:
    """Create prompt for JSON output generation"""

    names = [p['name'] for p in participants]

    prompt = f"""Analyze this meeting transcript and extract a structured JSON summary.

TRANSCRIPT:
{transcript[:6000]}

PARTICIPANTS: {', '.join(names)}

OUTPUT FORMAT (strict JSON):
{{
  "executiveSummary": "60-100 word summary: who met, purpose, key metrics, decisions, outcomes",
  "keyDecisions": ["decision 1", "decision 2", ...],
  "actionItems": [
    {{"task": "specific action", "assignee": "Name from list", "deadline": "timeframe", "priority": "high/medium/low"}}
  ],
  "nextSteps": ["step 1", "step 2", ...],
  "keyTopics": ["topic 1", "topic 2", ...],
  "sentiment": "{target_sentiment}"
}}

RULES:
- assignee MUST be one of: {', '.join(names)}
- sentiment MUST be: {target_sentiment}
- Include specific metrics mentioned in transcript
- 3-5 action items with varied priorities

Output valid JSON only:"""

    return prompt

print("Output prompt generator ready.")

Output prompt generator ready.


## Generation Functions

In [ ]:
def generate_batch(prompts: List[str], max_tokens: int, temperature: float = 0.8) -> List[str]:
    """Generate with higher temperature for diversity"""
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=0.92,
        max_tokens=max_tokens,
        repetition_penalty=1.08,
        stop=["</s>", "<|endoftext|>", "<|im_end|>"]
    )

    formatted = [
        f"<|im_start|>system\nYou transcribe meeting recordings. Output raw speech only, no labels.<|im_end|>\n<|im_start|>user\n{p}<|im_end|>\n<|im_start|>assistant\n"
        for p in prompts
    ]

    outputs = llm.generate(formatted, sampling_params)
    return [o.outputs[0].text.strip() for o in outputs]

print("Generation function ready.")

Generation function ready.


In [ ]:
def clean_transcript(transcript: str) -> str:
    """Remove any speaker labels that slipped through"""
    # Pattern: Name: or Name Surname:
    pattern = re.compile(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)?\s*:')
    cleaned = pattern.sub('', transcript)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

def extract_nlp_features(transcript: str) -> Dict[str, Any]:
    """Extract NLP features"""
    doc = nlp(transcript[:5000])

    entities = list(dict.fromkeys([
        f"{ent.text} ({ent.label_})" for ent in doc.ents
        if ent.label_ in ['PERSON', 'ORG', 'MONEY', 'PERCENT', 'DATE', 'CARDINAL']
    ]))[:15]

    phrases = list(dict.fromkeys([
        chunk.text.lower() for chunk in doc.noun_chunks
        if 2 <= len(chunk.text.split()) <= 4
    ]))[:10]

    nouns = [token.lemma_.lower() for token in doc if token.pos_ == "NOUN" and len(token.text) > 3]
    topics = [w for w, _ in Counter(nouns).most_common(6)]

    blob = TextBlob(transcript[:2000])
    polarity = round(blob.sentiment.polarity, 3)

    return {
        "entities": entities,
        "key_phrases": phrases,
        "topics": topics,
        "polarity": polarity,
        "sentiment_label": "Positive" if polarity > 0.1 else "Negative" if polarity < -0.05 else "Neutral"
    }

def format_input(transcript: str, nlp_features: Dict) -> str:
    """Format transcript with NLP features"""
    return f"""MEETING TRANSCRIPT:
{transcript}

NLP ANALYSIS:
Entities: {', '.join(nlp_features['entities']) or 'None'}
Key Phrases: {', '.join(nlp_features['key_phrases']) or 'None'}
Topics: {', '.join(nlp_features['topics']) or 'None'}
Sentiment: {nlp_features['sentiment_label']} (polarity: {nlp_features['polarity']})"""

print("Processing functions ready.")

Processing functions ready.


In [ ]:
def parse_json_output(response: str) -> Optional[Dict]:
    """Extract JSON from response"""
    response = response.strip()
    if response.startswith("```"):
        response = re.sub(r'^```[a-z]*\n?', '', response)
        response = re.sub(r'```$', '', response)

    match = re.search(r'\{[\s\S]*\}', response)
    if match:
        try:
            return json.loads(match.group())
        except:
            pass
    return None

def fix_output(output: Dict, participants: List[Dict], target_sentiment: str) -> Dict:
    """Fix common issues"""
    fixed = output.copy()
    names = [p['name'] for p in participants]

    if fixed.get('sentiment') not in ['positive', 'neutral', 'negative']:
        fixed['sentiment'] = target_sentiment

    for item in fixed.get('actionItems', []):
        if item.get('priority') not in ['high', 'medium', 'low']:
            item['priority'] = random.choice(['high', 'medium', 'low'])
        if not any(n in str(item.get('assignee', '')) for n in names):
            item['assignee'] = random.choice(names)
        if not item.get('deadline'):
            item['deadline'] = random.choice(DEADLINES)
        if not item.get('task') or len(item.get('task', '')) < 10:
            item['task'] = "Follow up on discussed action items"

    while len(fixed.get('actionItems', [])) < 2:
        fixed.setdefault('actionItems', []).append({
            'task': 'Follow up on meeting outcomes',
            'assignee': random.choice(names),
            'deadline': random.choice(DEADLINES),
            'priority': 'medium'
        })

    for key in ['keyDecisions', 'nextSteps', 'keyTopics']:
        if not fixed.get(key):
            fixed[key] = ["To be determined"]

    return fixed

def validate_output(output: Dict) -> bool:
    try:
        validate(instance=output, schema=OUTPUT_SCHEMA)
        return True
    except ValidationError:
        return False

print("Validation functions ready.")

Validation functions ready.


## Main Pipeline

In [ ]:
def generate_sample_batch(batch_specs: List[Dict], debug: bool = False) -> List[Dict]:
    """Generate samples with unique scenario seeds"""

    results = []

    # Generate unique seeds for each sample
    seeds = [generate_scenario_seed(spec['meeting_type'], spec['sentiment']) for spec in batch_specs]

    # Create prompts from seeds
    transcript_prompts = [create_transcript_prompt(seed) for seed in seeds]

    # Generate transcripts
    transcripts = generate_batch(transcript_prompts, config.transcript_max_tokens, temperature=0.85)

    # Validate and process
    output_prompts = []
    valid_indices = []

    for i, (transcript, seed) in enumerate(zip(transcripts, seeds)):
        transcript = clean_transcript(transcript)
        word_count = len(transcript.split())

        if debug:
            print(f"  [{seed['unique_id']}] {word_count} words")
            print(f"    Opening: {transcript[:100]}...")

        if config.min_transcript_words <= word_count <= config.max_transcript_words:
            transcripts[i] = transcript
            prompt = create_output_prompt(transcript, seed['participants'], seed['sentiment'])
            output_prompts.append(prompt)
            valid_indices.append(i)
        elif debug:
            print(f"    ❌ Invalid word count")

    if debug:
        print(f"  Valid: {len(valid_indices)}/{len(seeds)}")

    if not output_prompts:
        return results

    # Generate outputs
    outputs = generate_batch(output_prompts, config.output_max_tokens, temperature=0.3)

    # Process results
    for j, idx in enumerate(valid_indices):
        if j >= len(outputs):
            break

        transcript = transcripts[idx]
        seed = seeds[idx]

        parsed = parse_json_output(outputs[j])
        if not parsed:
            continue

        fixed = fix_output(parsed, seed['participants'], seed['sentiment'])
        if not validate_output(fixed):
            continue

        nlp_features = extract_nlp_features(transcript)
        input_text = format_input(transcript, nlp_features)

        results.append({
            'input': input_text,
            'output': json.dumps(fixed),
            'metadata': {
                'meeting_type': seed['meeting_type'],
                'sentiment': seed['sentiment'],
                'company': seed['company'],
                'transcript_words': len(transcript.split()),
                'seed_id': seed['unique_id']
            }
        })

    return results

print("Pipeline ready.")

Pipeline ready.


## Test - Check Diversity

In [ ]:
print("Testing diversity with 3 samples...")
print("="*80)
start = time.time()

test_specs = [
    {'sentiment': 'positive', 'meeting_type': 'quarterly_review'},
    {'sentiment': 'positive', 'meeting_type': 'quarterly_review'},
    {'sentiment': 'negative', 'meeting_type': 'performance_review'}
]

test_results = generate_sample_batch(test_specs, debug=True)
elapsed = time.time() - start

print(f"\n{'='*80}")
print(f"Generated {len(test_results)} samples in {elapsed:.1f}s")
print(f"{'='*80}")

# Show first 300 chars of each to compare diversity
print("\n🔍 DIVERSITY CHECK - First 300 chars of each transcript:")
print("="*80)
for i, sample in enumerate(test_results):
    meta = sample['metadata']
    transcript_start = sample['input'].split('\n\nNLP ANALYSIS')[0][:300]
    print(f"\n[{i+1}] {meta['meeting_type']} | {meta['company']} | {meta['sentiment']}")
    print(f"    {transcript_start}...")
    print()

Testing diversity with 3 samples...


Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  [63f4e479] 656 words
    Opening: Alright, good morning everyone, I hope you're all doing well today. You know, it’s been quite a roll...
    ❌ Invalid word count
  [97d79500] 905 words
    Opening: Sure, let’s jump into it. So last quarter we talked about increasing our market share by 10% and imp...
  [6d35de01] 690 words
    Opening: So let's kick things off, thanks Claire Moreau for starting us out. We’ve been going at this a while...
    ❌ Invalid word count
  Valid: 1/3


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Generated 1 samples in 22.5s

🔍 DIVERSITY CHECK - First 300 chars of each transcript:

[1] quarterly_review | SecureNet | positive
    MEETING TRANSCRIPT:
Sure, let’s jump into it. So last quarter we talked about increasing our market share by 10% and improving our conversion rates. Sophia, did you want to give us an update on where we stand? Well, I think we're doing alright but there are definitely some areas we need to focus on....



In [ ]:
# Full sample view
if test_results:
    print("\n" + "#"*80)
    print("FULL SAMPLE VIEW")
    print("#"*80)

    sample = test_results[0]
    meta = sample['metadata']
    output = json.loads(sample['output'])

    print(f"\nMeeting: {meta['meeting_type']} | Company: {meta['company']}")
    print(f"Sentiment: {meta['sentiment']} | Words: {meta['transcript_words']}")

    print(f"\n{'='*40} TRANSCRIPT {'='*40}")
    print(sample['input'])

    print(f"\n{'='*40} OUTPUT {'='*40}")
    print(f"\nSummary ({len(output['executiveSummary'].split())} words):")
    print(output['executiveSummary'])

    print(f"\nKey Decisions:")
    for d in output['keyDecisions']:
        print(f"  • {d}")

    print(f"\nAction Items:")
    for a in output['actionItems']:
        print(f"  • [{a['priority'].upper()}] {a['task']}")
        print(f"    → {a['assignee']} | {a['deadline']}")

    print(f"\nTopics: {', '.join(output['keyTopics'])}")
    print(f"Sentiment: {output['sentiment']}")


################################################################################
FULL SAMPLE VIEW
################################################################################

Meeting: quarterly_review | Company: SecureNet
Sentiment: positive | Words: 905

======================================== TRANSCRIPT ========================================
MEETING TRANSCRIPT:
Sure, let’s jump into it. So last quarter we talked about increasing our market share by 10% and improving our conversion rates. Sophia, did you want to give us an update on where we stand? Well, I think we're doing alright but there are definitely some areas we need to focus on. The current conversion rate is at 16%, which is slightly up from the last quarter, but it’s still not hitting our target. What do you think is holding us back here? I agree, let’s break down why. One thing that comes to mind is the complexity of our sales process. It might be overwhelming for potential clients. We have a lot of great offering

## Full Generation

In [ ]:
def create_generation_queue(total: int) -> List[Dict]:
    queue = []
    for sentiment, pct in config.sentiment_distribution.items():
        count = int(total * pct)
        for _ in range(count):
            queue.append({
                'sentiment': sentiment,
                'meeting_type': random.choice(config.meeting_types)
            })

    while len(queue) < total:
        queue.append({
            'sentiment': random.choices(
                list(config.sentiment_distribution.keys()),
                weights=list(config.sentiment_distribution.values())
            )[0],
            'meeting_type': random.choice(config.meeting_types)
        })

    random.shuffle(queue)
    return queue[:total]

target_with_buffer = int(config.total_samples * 2)
queue = create_generation_queue(target_with_buffer)
print(f"Queue: {len(queue)} specs for {config.total_samples} target")

Queue: 1000 specs for 500 target


In [ ]:
print("="*60)
print(f"GENERATING {config.total_samples} SAMPLES")
print("="*60)

all_samples = []
start_time = time.time()
batch_idx = 0
num_batches = (len(queue) + config.batch_size - 1) // config.batch_size

pbar = tqdm(total=config.total_samples, desc="Generating")

while len(all_samples) < config.total_samples and batch_idx < num_batches:
    batch_start = batch_idx * config.batch_size
    batch_end = min(batch_start + config.batch_size, len(queue))
    batch_specs = queue[batch_start:batch_end]

    try:
        results = generate_sample_batch(batch_specs)
        new_count = min(len(results), config.total_samples - len(all_samples))
        all_samples.extend(results[:new_count])
        pbar.update(new_count)
    except Exception as e:
        print(f"\n  Batch {batch_idx} error: {str(e)[:50]}")

    batch_idx += 1

    if batch_idx % 20 == 0:
        elapsed = time.time() - start_time
        rate = len(all_samples) / elapsed * 60
        print(f"\n  Progress: {len(all_samples)}/{config.total_samples} | {rate:.1f}/min")

pbar.close()
print(f"\n✅ Complete: {len(all_samples)} samples in {(time.time()-start_time)/60:.1f} min")

GENERATING 500 SAMPLES


Generating:   0%|          | 0/500 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

...

Adding requests:   0%|          | 0/16 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/16 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


✅ Complete: 500 samples in 25.7 min


In [ ]:
# Quality report
sentiment_counts = Counter()
meeting_counts = Counter()
word_counts = []

for s in all_samples:
    output = json.loads(s['output'])
    sentiment_counts[output['sentiment']] += 1
    meeting_counts[s['metadata']['meeting_type']] += 1
    word_counts.append(s['metadata']['transcript_words'])

print("\nQUALITY REPORT")
print("="*40)
print(f"Total: {len(all_samples)}")
print(f"\nSentiment:")
for s, c in sentiment_counts.most_common():
    print(f"  {s}: {c} ({c/len(all_samples)*100:.1f}%)")
print(f"\nTranscript words: {min(word_counts)}-{max(word_counts)}, avg={sum(word_counts)/len(word_counts):.0f}")


QUALITY REPORT
Total: 500

Sentiment:
  positive: 294 (58.8%)
  neutral: 126 (25.2%)
  negative: 80 (16.0%)

Transcript words: 700-1413, avg=825


In [ ]:
# Save
output_file = f"echonote_dataset_{len(all_samples)}.jsonl"

with open(output_file, 'w') as f:
    for s in all_samples:
        f.write(json.dumps({'input': s['input'], 'output': s['output']}) + '\n')

print(f"✅ Saved: {output_file}")

from google.colab import files
files.download(output_file)

✅ Saved: echonote_dataset_500.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>